### take the ICU clipped country shapes, buffer and convert from [-180 - 180] to [0 - 360]

In [29]:
%matplotlib inline

In [30]:
import os
import sys
import pathlib

In [31]:
import fiona
import geopandas as gpd

In [32]:
shapes_ipath = pathlib.Path.cwd().parents[1] / 'data' / 'shapefiles' / 'ICU' / 'clipped'

In [33]:
resources_path = pathlib.Path.cwd().parents[1] / 'resources'

In [34]:
shapes_opath = shapes_ipath / 'countries'

In [35]:
shapes_opath_converted = shapes_opath  / 'countries_converted'

In [36]:
shapefile_fname = 'Clipped_ICU_Countries2.shp'

In [37]:
shapefile = gpd.read_file(os.path.join(shapes_ipath, shapefile_fname))

In [24]:
def sanitize(country_name): 
    return country_name.replace(' ','_').replace(':','').replace('/','').replace('&','and')

### creates dictionnary mapping filenames to country names

In [25]:
dict_countries = {}

In [26]:
for country in shapefile.COUNTRYNAM.unique():
    dict_countries[sanitize(country)] = country

In [27]:
import json

with open(resources_path / 'dict_countries.json', 'w') as fp:
    json.dump(dict_countries, fp)

### loops and converts the shapes in the shapefile 

In [50]:
for country in shapefile.COUNTRYNAM.unique(): 
    country_fname = sanitize(country)
    shape_country = shapefile.query("COUNTRYNAM == '{}'".format(country))
    shape_country = shape_country.buffer(distance=0.1) 
    shape_country.to_file(os.path.join(shapes_opath, '{}'.format(country_fname))) 
    shapes = fiona.open(os.path.join(shapes_opath, '{}'.format(country_fname))) 
    if not os.path.exists(os.path.join(shapes_opath_converted, country_fname)): 
        os.mkdir(os.path.join(shapes_opath_converted, country_fname))
    with fiona.open(os.path.join(shapes_opath_converted, country_fname, 'shapes_{}.shp'.format(country_fname)), 'w', 'ESRI Shapefile', shapes.schema.copy(), crs = shapes.crs) as output:

        for shape in shapes:

    #         if shape['properties']['min_x'] < 0:

    #             shape['properties']['min_x'] += 360

    #         if shape['properties']['max_x'] < 0:

    #             shape['properties']['max_x'] += 360

            if (len(shape['geometry']['coordinates']) == 1):

                coords = shape['geometry']['coordinates']

                coords_mod = coords.copy()

                for j in range(len(coords[0])):

                    lon, lat = coords[0][j]

                    if lon < 0:

                        lon += 360

                    coords_mod[0][j] = (lon, lat)

                shape['geometry']['coordinates'] = coords_mod 

            else:

                for i in range(len(shape['geometry']['coordinates'])):

                    coords = shape['geometry']['coordinates'][i]

                    coords_mod = coords.copy()

                    for j in range(len(coords[0])):

                        lon, lat = coords[0][j]

                        if lon < 0:

                            lon += 360

                        coords_mod[0][j] = (lon, lat)

                    shape['geometry']['coordinates'][i] = coords_mod 

            output.write(shape)